In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout



In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


import os
for folder in [
    'data/train/with_mask',
    'data/train/without_mask',
    'data/test/with_mask',
    'data/test/without_mask'
    ]:
    os.makedirs(folder, exist_ok=True)

train_data = train_datagen.flow_from_directory('data/train', target_size=(128,128), batch_size=32, class_mode='categorical')
test_data = test_datagen.flow_from_directory('data/test', target_size=(128,128), batch_size=32, class_mode='categorical')


Found 7553 images belonging to 2 classes.
Found 7553 images belonging to 2 classes.
Found 7553 images belonging to 2 classes.


If you are still seeing an error, it is most likely because your folders `data/train` and `data/test` do not contain the required subfolders `with_mask` and `without_mask`, or those subfolders are empty.

To fix this, make sure your folder structure and image placement are exactly as follows:



In [ ]:
# data/
#   train/
#     with_mask/
#       image1.jpg
#       image2.jpg
#       ...
#     without_mask/
#       image1.jpg
#       image2.jpg
#       ...
#   test/
#     with_mask/
#       image1.jpg
#       image2.jpg
#       ...
#     without_mask/
#       image1.jpg
#       image2.jpg
#       ...



- Place your training images in `data/train/with_mask` and `data/train/without_mask`.
- Place your testing images in `data/test/with_mask` and `data/test/without_mask`.

If these folders are missing or empty, the code will still show an error.  
Let me know if you want a code cell to automatically create these folders for you!

In [5]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes: Mask, No Mask
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

c:\Users\Adithya\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,898 (12.61 MB)

 Trainable params: 3,304,898 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(train_data, validation_data=test_data, epochs=10)

model.save("mask_detector.h5")

c:\Users\Adithya\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
122/237 ━━━━━━━━━━━━━━━━━━━━ 56s 488ms/step - accuracy: 0.7043 - loss: 0.6020

c:\Users\Adithya\AppData\Local\Programs\Python\Python313\Lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


237/237 ━━━━━━━━━━━━━━━━━━━━ 159s 655ms/step - accuracy: 0.8390 - loss: 0.3719 - val_accuracy: 0.8949 - val_loss: 0.2449
Epoch 2/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 159s 655ms/step - accuracy: 0.8390 - loss: 0.3719 - val_accuracy: 0.8949 - val_loss: 0.2449
Epoch 2/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 143s 601ms/step - accuracy: 0.8939 - loss: 0.2593 - val_accuracy: 0.9232 - val_loss: 0.1871
Epoch 3/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 143s 601ms/step - accuracy: 0.8939 - loss: 0.2593 - val_accuracy: 0.9232 - val_loss: 0.1871
Epoch 3/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 142s 598ms/step - accuracy: 0.9187 - loss: 0.2017 - val_accuracy: 0.9364 - val_loss: 0.1571
Epoch 4/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 142s 598ms/step - accuracy: 0.9187 - loss: 0.2017 - val_accuracy: 0.9364 - val_loss: 0.1571
Epoch 4/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 145s 610ms/step - accuracy: 0.9318 - loss: 0.1777 - val_accuracy: 0.9376 - val_loss: 0.1689
Epoch 5/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 145s 610ms/step - accuracy: 0.9318 - loss: 0.17

In [7]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy*100:.2f}%")

237/237 ━━━━━━━━━━━━━━━━━━━━ 33s 140ms/step - accuracy: 0.9811 - loss: 0.0497
237/237 ━━━━━━━━━━━━━━━━━━━━ 33s 140ms/step - accuracy: 0.9811 - loss: 0.0497
Test Accuracy: 98.11%
Test Accuracy: 98.11%


In [10]:
# Load trained model
from tensorflow.keras.models import load_model
model = load_model("mask_detector.h5")

# Load face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (128,128))
        face = face / 255.0
        face = np.expand_dims(face, axis=0)

        pred = model.predict(face)
        label = "Mask" if np.argmax(pred)==0 else "No Mask"

        color = (0,255,0) if label=="Mask" else (0,0,255)
        cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow('Mask Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━